In [ ]:
from database import *
from backtest import *
import json

In [ ]:
with open('config.json') as f:
  config = json.load(f)

api = sj.Shioaji()
api.login(
    person_id=config['account'], 
    passwd=config['password'], 
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)

connection = sqlite3.connect('data.db')

# api.activate_ca(
#     ca_path=config['caPath'],
#     ca_passwd=config['caPassword'],
#     person_id=config['account'],
# )

In [ ]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

workers = {}
stop_threads = False

@api.quote.on_quote
def quote_callback(topic: str, quote: dict):
    
    code = topic.split('/')[-1]
    
    ts = pd.to_datetime(quote.get('Date') + ' ' + quote.get('Time'))
    
    if ts.hour < 9:
        return 
    
    close = quote.get('Close')[0]
    volume = quote.get('Volume')[0]
    vol_sum = quote.get('VolSum')[0]
    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum
    
    
def place_cb(stat, msg):
    
    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])

        order_qty[code] -= qty
        
        if order_qty[code] == 0:
            open_orders[code] = None
        
        position = positions.get(code, 0)
        
        if msg['action'] == 'Sell':
            qty = qty * -1
            
        position += qty
        
        positions[code] = position 

api.set_order_callback(place_cb)

In [ ]:
import time

def tbot(code):

    print('=== 啟動 tbot #{} ==='.format(code))
    
    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    if date not in tw_calendar.opens:
        print('今日非交易日')
        return 

    ticks[code] = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    ticks[code] = ticks[code][['close', 'volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='5Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()
    prev_close_price = kbars[code].loc[pd.to_datetime(prev_trading_date).replace(hour=13, minute=30)]['close']
    entry_price = -1
    first_close_price = -1
    first_upper_price = -1
    stop_loss = -1
    close_position = False
    api.quote.subscribe(api.Contracts.Stocks[code], quote_type='tick')
    
    while True:

        if stop_threads or prev_close_price is None:
            break
           
        time.sleep(1)
        
        current_time = datetime.datetime.now()
        
        if current_time.second == 0:

            kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='5Min'))
            kbars[code] = kbars[code][date:]
            
            str_current_time = (current_time - datetime.timedelta(minutes=5)).strftime('%Y-%m-%d %H:%M:%S')
            
            try:
                current_price = kbars[code].loc[str_current_time]['close']
                if first_close_price == -1:
                    str_open_time = current_time.replace(hour=9, minute=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
                    first_close_price = kbars[code].loc[str_open_time]['close']
                    first_upper_price = kbars[code].loc[str_open_time]['upper']
                    stop_loss = kbars[code].loc[str_open_time]['low']
                    
            except:
                continue
 
            open_order = open_orders.get(code, None)
            
            if open_order:
                
                api.update_status(api.stock_account)
                
                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:
                    if current_time > current_time.replace(hour=9, minute=10, second=0) and open_order.order.action == 'Buy':
                        api.cancel_order(open_order)
                        open_orders[code] = None
                        order_qty[code] = 0
                        if open_order.status.status != 'Filling':
                            break
                    elif current_time >= current_time.replace(hour=13, minute=20, second=0) and open_order.order.action == 'Sell':
                        api.cancel_order(open_order)
                        open_orders[code] = None
                        order_qty[code] = 0
#                         if open_order.status.status != 'Filling':
#                             break                                        
            position = positions.get(code, 0)
            
            if (
                position == 0 and not close_position and
                current_time <= current_time.replace(hour=9, minute=10, second=0) and
                prev_close_price * 1.05 <= first_close_price and
                prev_close_price * 1.09 > first_close_price and
                current_price > kbars[code].loc[str_current_time]['open'] and
                first_close_price > first_upper_price and
                current_price > kbars[code].loc[str_current_time]['upper'] and
                kbars[code].loc[str_current_time]['macdsignal'] > 0
            ):
                if current_price < 20:
                    qty = 10
                else:
                    qty = 1
                    
                contract = api.Contracts.Stocks[code]
                order = api.Order(
                    price=current_price,
                    quantity=qty,
                    action="Buy",
                    price_type="LMT",
                    order_type="ROD",
                    account=api.stock_account
                )
                    
                trade = api.place_order(contract, order)

                open_orders[code] = trade
                order_qty[code] = qty

                entry_price = current_price
                print('[{}] 委託買進 {} 價格 {}'.format(str_current_time, code, current_price))
                
            elif (
                position != 0 and not close_position and
                ((current_price >= entry_price * 1.03) or
                (current_price <= stop_loss) or
                (current_price > entry_price and kbars[code].loc[str_current_time]['rsi'] > 80) or
                (current_price >= entry_price * 1.01 and kbars[code].loc[str_current_time]['cci'] < -60) or 
                (current_price > entry_price and kbars[code].loc[str_current_time]['cci'] > 200))  
            ):
                
                contract = api.Contracts.Stocks[code]
                order = api.Order(
                    price=current_price,
                    quantity=position,
                    action="Sell",
                    price_type="LMT",
                    order_type="ROD",
                    account=api.stock_account
                )
                        
                trade = api.place_order(contract, order)
                
                close_position = True
                open_orders[code] = trade
                order_qty[code] = position
                        
                print('[{}] 委託賣出 {} 價格 {}'.format(str_current_time, code, current_price))
                    
            elif (
                current_time >= current_time.replace(hour=13, minute=20, second=0)
            ):
                if position != 0:
                    
                    contract = api.Contracts.Stocks[code]
                    order = api.Order(
                        price=0,
                        quantity=position,
                        action="Sell",
                        price_type="MKT",
                        order_type="ROD",
                        account=api.stock_account
                    )

                    trade = api.place_order(contract, order)
                    close_position = True
                    open_orders[code] = trade
                    order_qty[code] = position

                    print('[{}] 委託市價賣出 {} 市價 {}'.format(str_current_time, code, current_price))
                else:
                    break

                
    api.quote.unsubscribe(api.Contracts.Stocks[code], quote_type='tick')
    
    print('=== 關閉 tbot #{} ==='.format(code))


In [ ]:
import threading

def run():
    
    date = pd.to_datetime(datetime.datetime.now().date())
    codes = get_stocks(date, connection)    
    day_trading_codes = [code for code in codes if get_stock(code, connection, api)[0].iloc[0]['day_trade'] == 'Yes']
    print('今日當沖標的：{}'.format(day_trading_codes))
    
    for code in day_trading_codes:
        
        worker = threading.Thread(name=code, target=tbot, args=(code,))
        worker.start()
        workers[code] = worker

In [ ]:
update_historial_data('2021/5/5', '2021/5/5', connection, api)

In [ ]:
run()

In [ ]:
workers

In [ ]:
# ticks
kbars
# volume_today

In [ ]:
stop_threads = True